In [4]:
import numpy as np
import matplotlib.pyplot as plt
import threading
import queue
from scipy.special import erfc

# Tham số mô phỏng
num_bits = 8  # Số lượng bit dữ liệu gốc trong mỗi gói tin
polynomial = '1101'  # Đa thức CRC
timeout = 2  # Thời gian chờ ACK (Acknowledgment)
max_retransmissions = 5  # Số lần truyền lại tối đa

ack_received = threading.Event()
simulation_done = threading.Event()  # Sự kiện cho biết mô phỏng đã hoàn tất
BER_results = []

# Hàng đợi để truyền dữ liệu giữa sender và receiver
network_queue = queue.Queue()

# --- CRC ---
def crc_remainder(input_bitstring, polynomial_bitstring):
    polynomial_bitstring = polynomial_bitstring.lstrip('0')
    len_input = len(input_bitstring)
    input_padded_array = list(input_bitstring + '0' * (len(polynomial_bitstring) - 1))
    while '1' in input_padded_array[:len_input]:
        cur_shift = input_padded_array.index('1')
        for i in range(len(polynomial_bitstring)):
            input_padded_array[cur_shift + i] = str(int(polynomial_bitstring[i] != input_padded_array[cur_shift + i]))
    remainder = ''.join(input_padded_array)[len_input:]
    return remainder

def append_crc(data, polynomial=polynomial):
    data_bitstring = ''.join(map(str, data))
    crc = crc_remainder(data_bitstring, polynomial)
    data_with_crc = data_bitstring + crc
    return np.array([int(x) for x in data_with_crc])

def validate_crc(data_with_crc, polynomial=polynomial):
    data_with_crc_bitstring = ''.join(map(str, data_with_crc))
    remainder = crc_remainder(data_with_crc_bitstring, polynomial)
    return int(remainder) == 0

# --- Mã Hamming ---
def hamming_encode(data_with_crc):
    if len(data_with_crc) != 7:
        raise ValueError("Dữ liệu đầu vào phải có 7 bit (4 bit dữ liệu và 3 bit CRC).")
    # Hamming (7,4) Generator Matrix (G)
    G = np.array([[1, 1, 0, 1],
                  [1, 0, 1, 1],
                  [1, 0, 0, 0],
                  [0, 1, 1, 1],
                  [0, 1, 0, 0],
                  [0, 0, 1, 0],
                  [0, 0, 0, 1]])

    data_bits = data_with_crc[:4]  # 4 data bits
    crc_bits = data_with_crc[4:]   # 3 CRC bits
    encoded_data = np.dot(G, data_bits) % 2
    encoded_data = np.concatenate((encoded_data, crc_bits))  # Append CRC bits to encoded data
    return encoded_data

def hamming_decode(encoded_data_with_crc):
    # Extract encoded data and CRC bits
    encoded_data = encoded_data_with_crc[:7]  # 7 bits Hamming encoded data
    crc_bits = encoded_data_with_crc[7:]     # 3 bits CRC

    if len(encoded_data) != 7:
        raise ValueError("Dữ liệu đầu vào phải có 7 bit")
    
    # Hamming(7,4) Check Matrix (H)
    H = np.array([[1, 0, 1, 0, 1, 0, 1],
                  [0, 1, 1, 0, 0, 1, 1],
                  [0, 0, 0, 1, 1, 1, 1]])

    # Calculate the syndrome (error detection)
    syndrome = np.dot(H, encoded_data) % 2
    error_position = int(''.join(map(str, syndrome)), 2)  # Convert syndrome to error position

    # If there is an error (syndrome != 0), correct it
    if error_position != 0:
        encoded_data[error_position - 1] ^= 1  # Flip the bit at the error position (1-based index)

    # Extract the 4 data bits after correction
    data_bits = np.array([encoded_data[2], encoded_data[4], encoded_data[5], encoded_data[6]])
    dencoded_data = np.concatenate ([data_bits, crc_bits])

    return dencoded_data

# --- Chuyển đổi chéo ---
def interleave(codewords):
    interleaved = []
    for i in range(len(codewords[0])):
        interleaved.append([cw[i] for cw in codewords])
    return np.array(interleaved).flatten()

def deinterleave(codewords, original_length):
    deinterleaved = []
    for i in range(original_length):
        deinterleaved.append(codewords[i::original_length])
    return np.array(deinterleaved).flatten()

# --- Kênh truyền ---
def channel(tx_signal, SNR):
    noise = np.random.normal(0, np.sqrt(1/(2 * SNR)), tx_signal.shape)
    rx_signal = tx_signal + noise
    return np.where(rx_signal > 0.5, 1, 0)

# --- Hoạt động HARQ ---
def sender(data_packets):
    for sequence_number, packet in enumerate(data_packets):
        retransmission_count = 0
        while retransmission_count <= max_retransmissions:
            data_bits = packet[:4]  # Lấy 4 bit dữ liệu
            packet_with_crc = append_crc(data_bits)  # Thêm CRC vào gói tin
            encoded_packet = hamming_encode(packet_with_crc)  # Mã hóa gói tin với CRC
            interleaved_packet = interleave([encoded_packet])  # Xen kẽ gói tin
            
            # Đưa gói tin vào hàng đợi để gửi
            network_queue.put((sequence_number, interleaved_packet, packet))
            print(f"Sender: Gửi gói tin {sequence_number}, lần thứ {retransmission_count + 1}")
            
            ack_received.clear()  # Xóa trạng thái ACK cũ
            ack_received.wait(timeout=timeout)  # Chờ ACK trong khoảng thời gian timeout

            if ack_received.is_set():  # Nếu nhận được ACK
                break
            else:
                retransmission_count += 1
                print(f"Sender: Không nhận được ACK cho gói tin {sequence_number + 1}. Gửi lại gói tin.")
                
                if retransmission_count == max_retransmissions:
                    print(f"----Sender: Gửi lại gói tin {sequence_number + 1} quá số lần cho phép.----") 
    simulation_done.set()  # Đánh dấu mô phỏng đã hoàn tất


# --- Kết hợp gói tin (Chase Combining) ---
def chase_combine(previous_signals, current_signal):
    """
    Kết hợp các gói tin bằng cách tính giá trị trung bình của các bit từ các lần truyền trước và lần truyền hiện tại.
    """
    combined_signal = np.zeros_like(current_signal)
    for i in range(len(current_signal)):
        # Tính trung bình có trọng số của các bit nhận được từ các lần truyền trước
        combined_signal[i] = np.mean([previous_signals[j][i] for j in range(len(previous_signals))] + [current_signal[i]])
    return np.where(combined_signal > 0.5, 1, 0)


# --- Receiver with Chase Combining ---
def receiver_with_combining(data_packets, SNR):
    retransmissions = {}
    combined_signal = None  # Initialize the combined signal variable

    while True:
        if not network_queue.empty():
            sequence_number, encoded_packet, original_packet = network_queue.get()
            print(f"Receiver: Nhận gói tin {sequence_number}")

            # Perform channel decoding
            received_signal = channel(encoded_packet, SNR)
            deinterleaved_packet = deinterleave(received_signal, len(data_packets))

            # Perform Chase Combining: Combine current and previous received signals
            if combined_signal is None:
                combined_signal = deinterleaved_packet
            else:
                combined_signal = np.maximum(combined_signal, deinterleaved_packet)

            # Decoding after combining
            decoded_packet = hamming_decode(combined_signal)
            if validate_crc(decoded_packet):
                print(f"$ Receiver: Gửi ACK về Sender của gói tin {sequence_number} $")
                ack_received.set()
                combined_signal = None  # Reset after successful decoding
            else:
                print(f"Receiver: Gửi NACK về Sender của gói tin {sequence_number}")
                if sequence_number not in retransmissions:
                    retransmissions[sequence_number] = 0
                retransmissions[sequence_number] += 1

        if simulation_done.is_set():
            break

# --- Mô phỏng HARQ với Hamming, CRC và Chase Combining ---
def simulate_harq_hamming_crc_chase_combining(SNR):
    data_to_send = [np.random.randint(0, 2, num_bits) for _ in range(5)]
    sender_thread = threading.Thread(target=sender, args=(data_to_send,))
    receiver_thread = threading.Thread(target=receiver_with_combining, args=(data_to_send,SNR))

    receiver_thread.start()
    sender_thread.start()

    sender_thread.join()
    receiver_thread.join()
simulate_harq_hamming_crc_chase_combining(1)


Sender: Gửi gói tin 0, lần thứ 1
Receiver: Nhận gói tin 0
Receiver: Gửi NACK về Sender của gói tin 0
Sender: Không nhận được ACK cho gói tin 1. Gửi lại gói tin.
Sender: Gửi gói tin 0, lần thứ 2
Receiver: Nhận gói tin 0
Receiver: Gửi NACK về Sender của gói tin 0
Sender: Không nhận được ACK cho gói tin 1. Gửi lại gói tin.
Sender: Gửi gói tin 0, lần thứ 3
Receiver: Nhận gói tin 0
Receiver: Gửi NACK về Sender của gói tin 0
Sender: Không nhận được ACK cho gói tin 1. Gửi lại gói tin.
Sender: Gửi gói tin 0, lần thứ 4
Receiver: Nhận gói tin 0
$ Receiver: Gửi ACK về Sender của gói tin 0 $
Sender: Gửi gói tin 1, lần thứ 1
Receiver: Nhận gói tin 1
Receiver: Gửi NACK về Sender của gói tin 1
Sender: Không nhận được ACK cho gói tin 2. Gửi lại gói tin.
Sender: Gửi gói tin 1, lần thứ 2
Receiver: Nhận gói tin 1
Receiver: Gửi NACK về Sender của gói tin 1
Sender: Không nhận được ACK cho gói tin 2. Gửi lại gói tin.
Sender: Gửi gói tin 1, lần thứ 3
Receiver: Nhận gói tin 1
Receiver: Gửi NACK về Sender của 

In [ ]:
import numpy as np
import threading
import queue
from scipy.special import erfc

# Tham số mô phỏng
num_bits = 12  # Số lượng bit dữ liệu gốc trong mỗi gói tin
polynomial = '1101'  # Đa thức CRC
timeout = 2  # Thời gian chờ ACK (Acknowledgment)
max_retransmissions = 5  # Số lần truyền lại tối đa

ack_received = threading.Event()
simulation_done = threading.Event()  # Sự kiện cho biết mô phỏng đã hoàn tất
BER_results = []

# Hàng đợi để truyền dữ liệu giữa sender và receiver
network_queue = queue.Queue()

# --- CRC ---
def crc_remainder(input_bitstring, polynomial_bitstring):
    polynomial_bitstring = polynomial_bitstring.lstrip('0')
    len_input = len(input_bitstring)
    input_padded_array = list(input_bitstring + '0' * (len(polynomial_bitstring) - 1))
    while '1' in input_padded_array[:len_input]:
        cur_shift = input_padded_array.index('1')
        for i in range(len(polynomial_bitstring)):
            input_padded_array[cur_shift + i] = str(int(polynomial_bitstring[i] != input_padded_array[cur_shift + i]))
    remainder = ''.join(input_padded_array)[len_input:]
    return remainder

def append_crc(data, polynomial=polynomial):
    data_bitstring = ''.join(map(str, data))
    crc = crc_remainder(data_bitstring, polynomial)
    data_with_crc = data_bitstring + crc
    return np.array([int(x) for x in data_with_crc])

def validate_crc(data_with_crc, polynomial=polynomial):
    data_with_crc_bitstring = ''.join(map(str, data_with_crc))
    remainder = crc_remainder(data_with_crc_bitstring, polynomial)
    return int(remainder) == 0

# --- Mã Hamming ---
def hamming_encode(data_with_crc):
    if len(data_with_crc) != 7:
        raise ValueError("Dữ liệu đầu vào phải có 7 bit (4 bit dữ liệu và 3 bit CRC).")
    # Hamming (7,4) Generator Matrix (G)
    G = np.array([[1, 1, 0, 1],
                  [1, 0, 1, 1],
                  [1, 0, 0, 0],
                  [0, 1, 1, 1],
                  [0, 1, 0, 0],
                  [0, 0, 1, 0],
                  [0, 0, 0, 1]])

    data_bits = data_with_crc[:4]  # 4 data bits
    crc_bits = data_with_crc[4:]   # 3 CRC bits
    encoded_data = np.dot(G, data_bits) % 2
    encoded_data = np.concatenate((encoded_data, crc_bits))  # Append CRC bits to encoded data
    return encoded_data

def hamming_decode(encoded_data_with_crc):
    # Extract encoded data and CRC bits
    encoded_data = encoded_data_with_crc[:7]  # 7 bits Hamming encoded data
    crc_bits = encoded_data_with_crc[7:]     # 3 bits CRC

    if len(encoded_data) != 7:
        raise ValueError("Dữ liệu đầu vào phải có 7 bit")
    
    # Hamming(7,4) Check Matrix (H)
    H = np.array([[1, 0, 1, 0, 1, 0, 1],
                  [0, 1, 1, 0, 0, 1, 1],
                  [0, 0, 0, 1, 1, 1, 1]])

    # Calculate the syndrome (error detection)
    syndrome = np.dot(H, encoded_data) % 2
    error_position = int(''.join(map(str, syndrome)), 2)  # Convert syndrome to error position

    # If there is an error (syndrome != 0), correct it
    if error_position != 0:
        encoded_data[error_position - 1] ^= 1  # Flip the bit at the error position (1-based index)

    # Extract the 4 data bits after correction
    data_bits = np.array([encoded_data[2], encoded_data[4], encoded_data[5], encoded_data[6]])
    decoded_data = np.concatenate ([data_bits, crc_bits])

    return decoded_data

# --- Chuyển đổi chéo ---
def interleave(codewords):
    interleaved = []
    for i in range(len(codewords[0])):
        interleaved.append([cw[i] for cw in codewords])
    return np.array(interleaved).flatten()

def deinterleave(codewords, original_length):
    deinterleaved = []
    for i in range(original_length):
        deinterleaved.append(codewords[i::original_length])
    return np.array(deinterleaved).flatten()

# --- Kênh truyền ---
def channel(tx_signal, SNR):
    noise = np.random.normal(0, np.sqrt(1/(2 * SNR)), tx_signal.shape)
    rx_signal = tx_signal + noise
    return np.where(rx_signal > 0.5, 1, 0)

# --- Hoạt động HARQ ---
def sender(data_packets):
    for sequence_number, packet in enumerate(data_packets):
        retransmission_count = 0
        while retransmission_count <= max_retransmissions:
            data_bits = packet[:4]  # Lấy 4 bit dữ liệu
            packet_with_crc = append_crc(data_bits)  # Thêm CRC vào gói tin
            encoded_packet = hamming_encode(packet_with_crc)  # Mã hóa gói tin với CRC
            interleaved_packet = interleave([encoded_packet])  # Xen kẽ gói tin
            
            # Đưa gói tin vào hàng đợi để gửi
            network_queue.put((sequence_number, interleaved_packet, packet))
            print(f"Sender: Gửi gói tin {sequence_number}")
            
            ack_received.clear()
            ack_received.wait(timeout=timeout)

            if ack_received.is_set():
                break
            else:
                retransmission_count += 1
                print(f"Sender: Không nhận được ACK cho gói tin {sequence_number}. Gửi lại gói tin.")
    simulation_done.set()

# --- Kết hợp gói tin (Chase Combining) ---
def chase_combine(previous_signals, current_signal):
    """
    Kết hợp các gói tin bằng cách tính giá trị trung bình của các bit từ các lần truyền trước và lần truyền hiện tại.
    """
    combined_signal = np.zeros_like(current_signal)
    for i in range(len(current_signal)):
        # Tính trung bình có trọng số của các bit nhận được từ các lần truyền trước
        combined_signal[i] = np.mean([previous_signals[j][i] for j in range(len(previous_signals))] + [current_signal[i]])
    return np.where(combined_signal > 0.5, 1, 0)


# --- Receiver with Chase Combining ---
def receiver_with_combining(data_packets, SNR, retransmissions, max_retransmissions):
    retransmission_count = {}  # Đếm số lần truyền lại cho từng gói tin
    for sequence_number, encoded_packet in enumerate(data_packets):
        combined_signal = None  # Biến tín hiệu kết hợp cho mỗi gói tin
        
        # Tiến hành quá trình nhận và xử lý gói tin cho từng sequence_number
        for n in range(max_retransmissions):
            print(f"Receiver: Đang nhận gói tin {sequence_number} lần truyền {n+1}")
            
            # Giải mã tín hiệu
            received_signal = channel(encoded_packet, SNR)
            deinterleaved_packet = deinterleave(received_signal, len(data_packets))

            # Chase Combining: Kết hợp tín hiệu đã nhận
            if combined_signal is None:
                combined_signal = deinterleaved_packet
            else:
                combined_signal = np.maximum(combined_signal, deinterleaved_packet)

            # Giải mã sau khi kết hợp
            decoded_packet = hamming_decode(combined_signal)

            # Kiểm tra tính hợp lệ của gói tin qua CRC
            if validate_crc(decoded_packet):
                print(f"Receiver: Gửi ACK về Sender cho gói tin {sequence_number} thành công ở lần truyền {n+1}")
                ack_received.set()
                break  # Nếu gói tin nhận thành công, dừng lại
            else:
                print(f"Receiver: Gửi NACK về Sender cho gói tin {sequence_number} ở lần truyền {n+1}")
                retransmission_count[sequence_number] = retransmission_count.get(sequence_number, 0) + 1
                
                if retransmission_count[sequence_number] >= max_retransmissions:
                    print(f"Gói tin {sequence_number} đã vượt quá số lần truyền lại tối đa.")
                    break  # Dừng lại nếu vượt quá số lần truyền lại tối đa

        # Đảm bảo cập nhật số lần truyền lại vào danh sách retransmissions
        if retransmission_count[sequence_number] > 0:
            retransmissions[sequence_number] = retransmission_count[sequence_number]

# --- Hàm tính toán ---
def calculate_PF(SNR):
    """
    Tính xác suất lỗi PF từ SNR.
    """
    return 1 - np.exp(-SNR)  # Giả sử giá trị này theo mô hình đơn giản

# --- Simulation ---
def simulate_harq_hamming_crc_chase_combining(SNR, max_retransmissions=5):
    """
    Mô phỏng HARQ với Hamming, CRC và Chase Combining.
    """
    data_to_send = [np.random.randint(0, 2, num_bits) for _ in range(5)]  # Gửi 5 gói dữ liệu
    retransmissions = {}  # Đếm số lần truyền lại cho từng gói dữ liệu
    sender_thread = threading.Thread(target=sender, args=(data_to_send,))
    receiver_thread = threading.Thread(target=receiver_with_combining, args=(data_to_send, SNR, retransmissions, max_retransmissions))

    receiver_thread.start()
    sender_thread.start()

    sender_thread.join()
    receiver_thread.join()

    total_bits = num_bits * len(data_to_send)
    bit_errors = 0

    # Tính số lỗi bit trong dữ liệu nhận được
    while not network_queue.empty():
        sequence_number, received_packet, original_packet = network_queue.get()
        if received_packet is not None:
            received_data = received_packet[:4]  # Lấy 4 bit đầu tiên từ gói tin nhận
            bit_errors += np.count_nonzero(received_data != original_packet[:4])  # Tính lỗi bit
    total_bit_errors = bit_errors
    return total_bit_errors / total_bits, retransmissions

# --- Kết quả ---
if __name__ == "__main__":
    for SNR in [1, 3, 5, 10, 15, 20]:
        bit_error_rate, retransmissions = simulate_harq_hamming_crc_chase_combining(SNR)
        print(f"SNR = {SNR}dB, Tỉ lệ lỗi bit (BER) = {bit_error_rate:.4f}, Số lần truyền lại: {retransmissions}")


Exception in thread Thread-4:
Traceback (most recent call last):
  File "c:\Users\Admin\miniconda3\envs\tf-env\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "c:\Users\Admin\miniconda3\envs\tf-env\lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\Admin\miniconda3\envs\tf-env\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Admin\AppData\Local\Temp\ipykernel_4972\1388200971.py", line 155, in receiver_with_combining
  File "C:\Users\Admin\AppData\Local\Temp\ipykernel_4972\1388200971.py", line 99, in deinterleave
ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (5,) + inhomogeneous part.


Receiver: Đang nhận gói tin 0 lần truyền 1
Sender: Gửi gói tin 0
Sender: Không nhận được ACK cho gói tin 0. Gửi lại gói tin.
Sender: Gửi gói tin 0
Sender: Không nhận được ACK cho gói tin 0. Gửi lại gói tin.
Sender: Gửi gói tin 0
Sender: Không nhận được ACK cho gói tin 0. Gửi lại gói tin.
Sender: Gửi gói tin 0
Sender: Không nhận được ACK cho gói tin 0. Gửi lại gói tin.
Sender: Gửi gói tin 0
Sender: Không nhận được ACK cho gói tin 0. Gửi lại gói tin.
Sender: Gửi gói tin 0
Sender: Không nhận được ACK cho gói tin 0. Gửi lại gói tin.
Sender: Gửi gói tin 1
Sender: Không nhận được ACK cho gói tin 1. Gửi lại gói tin.
Sender: Gửi gói tin 1
Sender: Không nhận được ACK cho gói tin 1. Gửi lại gói tin.
Sender: Gửi gói tin 1
Sender: Không nhận được ACK cho gói tin 1. Gửi lại gói tin.
Sender: Gửi gói tin 1
Sender: Không nhận được ACK cho gói tin 1. Gửi lại gói tin.
Sender: Gửi gói tin 1
Sender: Không nhận được ACK cho gói tin 1. Gửi lại gói tin.
Sender: Gửi gói tin 1
Sender: Không nhận được ACK cho g

Exception in thread Thread-6:
Traceback (most recent call last):
  File "c:\Users\Admin\miniconda3\envs\tf-env\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "c:\Users\Admin\miniconda3\envs\tf-env\lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\Admin\miniconda3\envs\tf-env\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Admin\AppData\Local\Temp\ipykernel_4972\1388200971.py", line 155, in receiver_with_combining
  File "C:\Users\Admin\AppData\Local\Temp\ipykernel_4972\1388200971.py", line 99, in deinterleave
ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (5,) + inhomogeneous part.


Sender: Không nhận được ACK cho gói tin 4. Gửi lại gói tin.
SNR = 1dB, Tỉ lệ lỗi bit (BER) = 0.9000, Số lần truyền lại: {}
Receiver: Đang nhận gói tin 0 lần truyền 1
Sender: Gửi gói tin 0
Sender: Không nhận được ACK cho gói tin 0. Gửi lại gói tin.
Sender: Gửi gói tin 0
Sender: Không nhận được ACK cho gói tin 0. Gửi lại gói tin.
Sender: Gửi gói tin 0
Sender: Không nhận được ACK cho gói tin 0. Gửi lại gói tin.
Sender: Gửi gói tin 0
Sender: Không nhận được ACK cho gói tin 0. Gửi lại gói tin.
Sender: Gửi gói tin 0
Sender: Không nhận được ACK cho gói tin 0. Gửi lại gói tin.
Sender: Gửi gói tin 0
Sender: Không nhận được ACK cho gói tin 0. Gửi lại gói tin.
Sender: Gửi gói tin 1


In [ ]:

# # --- Vẽ đồ thị Thông lượng và Độ trễ theo SNR ---
# def plot_throughput_delay_snr():
#     SNR_dB_values = range(0, 15, 1)
#     throughput_results = []
#     delay_results = []

#     for SNR_dB in SNR_dB_values:
#         SNR = 10**(SNR_dB / 10)
#         Tp, D = simulate_harq_hamming_crc(SNR)
#         throughput_results.append(Tp)
#         delay_results.append(D)

#     plt.figure()
#     plt.subplot(2, 1, 1)
#     plt.plot(SNR_dB_values, throughput_results, marker='o', label='Thông lượng')
#     plt.xlabel('SNR (dB)')
#     plt.ylabel('Thông lượng (T_p)')
#     plt.title('Thông lượng theo SNR')
#     plt.grid(True)

#     plt.subplot(2, 1, 2)
#     plt.plot(SNR_dB_values, delay_results, marker='o', label='Độ trễ trung bình')
#     plt.xlabel('SNR (dB)')
#     plt.ylabel('Độ trễ trung bình (D)')
#     plt.title('Độ trễ theo SNR')
#     plt.grid(True)

#     plt.tight_layout()
#     plt.show()

# # Chạy mô phỏng và vẽ đồ thị
# plot_throughput_delay_snr()


In [2]:
# --- CRC ---
import numpy as np
def crc_remainder(input_bitstring, polynomial_bitstring):
    polynomial_bitstring = polynomial_bitstring.lstrip('0')
    len_input = len(input_bitstring)
    input_padded_array = list(input_bitstring + '0' * (len(polynomial_bitstring) - 1))
    while '1' in input_padded_array[:len_input]:
        cur_shift = input_padded_array.index('1')
        for i in range(len(polynomial_bitstring)):
            input_padded_array[cur_shift + i] = str(int(polynomial_bitstring[i] != input_padded_array[cur_shift + i]))
    remainder = ''.join(input_padded_array)[len_input:]
    return remainder

def append_crc(data, polynomial):
    data_bitstring = ''.join(map(str, data))
    crc = crc_remainder(data_bitstring, polynomial)
    data_with_crc = data_bitstring + crc
    return np.array([int(x) for x in data_with_crc])

def validate_crc(data_with_crc, polynomial):
    data_with_crc_bitstring = ''.join(map(str, data_with_crc))
    remainder = crc_remainder(data_with_crc_bitstring, polynomial)
    return int(remainder) == 0

data = [1, 0, 1, 0, 0, 0, 0]
polynomial = '1001'

data_with_crc = append_crc(data, polynomial=polynomial)
print("Data with CRC:", data_with_crc)  # Output: [1 0 1 0 0 0 0 0 1 1]
is_valid = validate_crc(data_with_crc, polynomial)
print("Is valid:", is_valid)

Data with CRC: [1 0 1 0 0 0 0 0 1 1]
Is valid: True


In [86]:
import numpy as np

# --- Mã Hamming ---
def hamming_encode(data_with_crc):
    if len(data_with_crc) != 7:
        raise ValueError("Dữ liệu đầu vào phải có 7 bit (4 bit dữ liệu và 3 bit CRC).")
    # Hamming (7,4) Generator Matrix (G)
    G = np.array([[1, 1, 0, 1],
                  [1, 0, 1, 1],
                  [1, 0, 0, 0],
                  [0, 1, 1, 1],
                  [0, 1, 0, 0],
                  [0, 0, 1, 0],
                  [0, 0, 0, 1]])

    data_bits = data_with_crc[:4]  # 4 data bits
    crc_bits = data_with_crc[4:]   # 3 CRC bits
    encoded_data = np.dot(G, data_bits) % 2
    encoded_data = np.concatenate((encoded_data, crc_bits))  # Append CRC bits to encoded data
    return encoded_data

def hamming_decode(encoded_data_with_crc):
    # Extract encoded data and CRC bits
    encoded_data = encoded_data_with_crc[:7]  # 7 bits Hamming encoded data
    crc_bits = encoded_data_with_crc[7:]     # 3 bits CRC

    if len(encoded_data) != 7:
        raise ValueError("Dữ liệu đầu vào phải có 7 bit")
    
    # Hamming(7,4) Check Matrix (H)
    H = np.array([[1, 0, 1, 0, 1, 0, 1],
                  [0, 1, 1, 0, 0, 1, 1],
                  [0, 0, 0, 1, 1, 1, 1]])

    # Calculate the syndrome (error detection)
    syndrome = np.dot(H, encoded_data) % 2
    error_position = int(''.join(map(str, syndrome)), 2)  # Convert syndrome to error position

    # If there is an error (syndrome != 0), correct it
    if error_position != 0:
        encoded_data[error_position - 1] ^= 1  # Flip the bit at the error position (1-based index)

    # Extract the 4 data bits after correction
    data_bits = np.array([encoded_data[2], encoded_data[4], encoded_data[5], encoded_data[6]])
    dencoded_data = np.concatenate ([data_bits, crc_bits])

    return dencoded_data

# Test
data = [1, 0, 1, 1]  # 4 data bits
crc = [1, 0, 1]      # 3 CRC bits
data_with_crc = np.concatenate([data, crc])  # Combine data and CRC into 7 bits
print(f"Data with CRC: {data_with_crc}")

encoded_data = hamming_encode(data_with_crc)  # Encode using Hamming
print(f"Encoded data: {encoded_data}")

# Simulate transmission with an error (flip a bit)
encoded_data_with_error = encoded_data.copy()
encoded_data_with_error[3] ^= 1  # Introduce an error at position 4 (index 3)

print(f"Encoded data with error: {encoded_data_with_error}")

# Decode the received data and correct any errors
decoded_data, decoded_crc = hamming_decode(encoded_data_with_error)
print(f"Decoded data: {decoded_data}, Decoded CRC: {decoded_crc}")


Data with CRC: [1 0 1 1 1 0 1]
Encoded data: [0 1 1 0 0 1 1 1 0 1]
Encoded data with error: [0 1 1 1 0 1 1 1 0 1]
Decoded data: [1 0 1 1], Decoded CRC: [1 0 1]


In [7]:
import numpy as np

def hamming_encode_71_64(input_bits):
    """
    Mã hóa dữ liệu với mã Hamming (71, 64).

    Args:
        input_bits (numpy array): Mảng numpy chứa 64 bit dữ liệu đầu vào.

    Returns:
        numpy array: Chuỗi 71 bit đã mã hóa.
    """
    if len(input_bits) != 64:
        raise ValueError("Dữ liệu đầu vào phải có 64 bit.")

    encoded_bits = np.zeros(71, dtype=int)
    data_index = 0

    # Chèn dữ liệu vào các vị trí không phải là 2^i
    for i in range(71):
        if (i + 1) & i != 0:  # Các vị trí không phải là 2^i
            encoded_bits[i] = input_bits[data_index]
            data_index += 1

    # Tính toán các bit kiểm tra tại các vị trí 2^i
    for i in range(7):  # Các vị trí kiểm tra tại 1, 2, 4, 8, 16, 32, 64
        parity_pos = 2**i - 1
        parity_value = 0
        for j in range(71):
            if (j + 1) & (parity_pos + 1) != 0:
                parity_value ^= encoded_bits[j]
        encoded_bits[parity_pos] = parity_value

    return encoded_bits


def hamming_decode_71_64(received_bits):
    """
    Giải mã mã Hamming (71, 64) và sửa lỗi nếu phát hiện.

    Args:
        received_bits (numpy array): Mảng numpy chứa 71 bit dữ liệu đã nhận.

    Returns:
        numpy array: Chuỗi dữ liệu sau khi giải mã (64 bit).
        int: Vị trí lỗi (nếu có, 0 nếu không có lỗi).
    """
    if len(received_bits) != 71:
        raise ValueError("Dữ liệu đầu vào phải có 71 bit.")

    # Ma trận kiểm tra H (7 x 71)
    H = np.array([[int(bit) for bit in format(i + 1, '07b')] for i in range(71)]).T

    # Tính hội chứng (syndrome)
    syndrome = np.dot(H, received_bits) % 2
    error_position = int(''.join(map(str, syndrome)), 2)  # Chuyển hội chứng sang vị trí lỗi

    # Nếu có lỗi (error_position != 0), sửa lỗi
    corrected_bits = received_bits.copy()
    if error_position != 0:
        corrected_bits[error_position - 1] ^= 1  # Lật bit tại vị trí lỗi

    # Loại bỏ các bit kiểm tra (chỉ lấy 64 bit dữ liệu)
    decoded_bits = []
    for i in range(71):
        if (i + 1) & i != 0:  # Loại bỏ các vị trí 2^i
            decoded_bits.append(corrected_bits[i])

    return np.array(decoded_bits), error_position


# --- Ví dụ sử dụng ---
if __name__ == "__main__":
    # Tạo dữ liệu 64 bit
    data_bits = np.random.randint(0, 2, 64)

    print(f"Dữ liệu gốc (64 bit): {data_bits}")

    # Encode
    encoded_bits = hamming_encode_71_64(data_bits)
    print(f"Dữ liệu sau mã hóa (71 bit): {encoded_bits}")

    # Tạo lỗi tại một vị trí (ví dụ: vị trí 20)
    received_bits = encoded_bits.copy()
    received_bits[19] ^= 1  # Tạo lỗi tại vị trí 20 (1-based index)
    print(f"Dữ liệu nhận được (có lỗi): {received_bits}")

    # Decode
    decoded_bits, error_position = hamming_decode_71_64(received_bits)
    print(f"Dữ liệu sau giải mã (64 bit): {decoded_bits}")
    if error_position == 0:
        print("Không phát hiện lỗi.")
    else:
        print(f"Đã phát hiện lỗi và sửa tại vị trí: {error_position}")

    # Kiểm tra dữ liệu gốc và dữ liệu sau giải mã
    print(f"Giải mã thành công: {np.array_equal(data_bits, decoded_bits)}")


Dữ liệu gốc (64 bit): [1 0 0 1 1 0 1 1 0 0 0 1 1 0 1 0 1 1 0 0 0 1 0 1 1 1 0 0 1 0 1 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 1 1 1 0 0 0 1 1 1 0 0 0 1 0 0 0 0 0 0]
Dữ liệu sau mã hóa (71 bit): [0 0 1 0 0 0 1 1 1 0 1 1 0 0 0 1 1 1 0 1 0 1 1 0 0 0 1 0 1 1 1 1 0 0 1 0 1
 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 1 0 0 0 1 1 1 0 0 0 1 1 0 0 0 0 0 0]
Dữ liệu nhận được (có lỗi): [0 0 1 0 0 0 1 1 1 0 1 1 0 0 0 1 1 1 0 0 0 1 1 0 0 0 1 0 1 1 1 1 0 0 1 0 1
 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 1 0 0 0 1 1 1 0 0 0 1 1 0 0 0 0 0 0]
Dữ liệu sau giải mã (64 bit): [1 0 0 1 1 0 1 1 0 0 0 1 1 0 1 0 1 1 0 0 0 1 0 1 1 1 0 0 1 0 1 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 1 1 1 0 0 0 1 1 1 0 0 0 1 0 0 0 0 0 0]
Đã phát hiện lỗi và sửa tại vị trí: 20
Giải mã thành công: True


In [9]:
data_to_send = [np.random.randint(0, 2, 64) for _ in range(32)]
data_to_send

[array([0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1,
        0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0]),
 array([0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0,
        1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1,
        1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0]),
 array([0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1,
        1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1,
        0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1]),
 array([1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1,
        0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0,
        1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1]),
 array([1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0,
        1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1

In [12]:
import random

# Dữ liệu gốc
encoded_data = [1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1]

# Mô phỏng lỗi bit
error = 0.1
corrupted_data = []

for bit in encoded_data:
    corrupted_bit = bit
    # Giữ nguyên giá trị bit (không cần chuyển thành chuỗi nhị phân)
    if random.random() < error:
        # Lật bit nếu cần
        corrupted_bit = 1 - bit  # Nếu bit = 0 thì đổi thành 1, nếu bit = 1 thì đổi thành 0
        
    corrupted_data.append(corrupted_bit)

# So sánh chuỗi gốc và chuỗi bị lỗi
differences = [(original, corrupted) for original, corrupted in zip(encoded_data, corrupted_data) if original != corrupted]

differences


[(1, 0), (1, 0), (1, 0), (1, 0)]

In [43]:
class TurboCode:
    def __init__(self):
        self.generator_1 = [1, 0, 1]  # Polynomial G1(x) = 1 + x^2
        self.generator_2 = [1, 1, 1]  # Polynomial G2(x) = 1 + x + x^2

    def convolutional_encode(self, data, generator):
        n = len(generator)
        encoded = []
        register = [0] * n
        for bit in data:
            register = [bit] + register[:-1]  # Shift register
            encoded_bit = sum([register[i] * generator[i] for i in range(n)]) % 2
            encoded.append(encoded_bit)
        return encoded

    def turbo_encode(self, data):
        systematic_bits = data
        parity_bits_1 = self.convolutional_encode(data, self.generator_1)
        interleaved_data = np.random.permutation(data)  # Hoán vị
        parity_bits_2 = self.convolutional_encode(interleaved_data, self.generator_2)
        encoded_data = list(zip(systematic_bits, parity_bits_1, parity_bits_2))
        return encoded_data, interleaved_data

    def puncture(self, encoded_data, puncture_pattern):
        punctured_data = []
        for i, bits in enumerate(encoded_data):
            punctured_bits = [bits[j] for j in range(len(bits)) if puncture_pattern[j] == 1]
            punctured_data.append(punctured_bits)
        return punctured_data

    def turbo_decode(self, received_data, interleaved_data, max_iterations=10):
        decoded_data = [bits[0] for bits in received_data]  # Bit hệ thống
        confidence = [1.0] * len(decoded_data)  # Trọng số tin cậy ban đầu

        for _ in range(max_iterations):
            errors = []
            for i, bits in enumerate(received_data):
                if len(bits) == 3:  # Kiểm tra đầy đủ hệ thống, P1, P2
                    system_bit, parity_1, parity_2 = bits
                    check_p1 = parity_1 == self.convolutional_encode([system_bit], self.generator_1)[0]
                    check_p2 = parity_2 == self.convolutional_encode([interleaved_data[i]], self.generator_2)[0]

                    if not (check_p1 and check_p2):
                        errors.append(i)

            for error_index in errors:
                confidence[error_index] *= 0.5  # Giảm tin cậy cho bit lỗi
                if confidence[error_index] < 0.5:  # Khi tin cậy thấp, đảo bit
                    decoded_data[error_index] = 1 - decoded_data[error_index]

            if not errors:  # Nếu không còn lỗi, dừng giải mã
                break

        return decoded_data


# --- Ví dụ sử dụng ---
data_str = "111011"  # Dữ liệu gốc dưới dạng chuỗi
data = list(map(int, data_str))  # Chuyển chuỗi thành danh sách bit
print("Dữ liệu gốc:", data)

# Tạo bộ mã Turbo
turbo = TurboCode()

# Mã hóa Turbo
encoded_data, interleaved_data = turbo.turbo_encode(data)
print("Dữ liệu mã hóa Turbo (1/3):", encoded_data)

# Đục lỗ (puncturing) để giảm tỷ lệ mã hóa
puncture_pattern = [1, 0, 1] 
punctured_data = turbo.puncture(encoded_data, puncture_pattern)
print("Dữ liệu sau khi đục lỗ:", punctured_data)

# Giả sử truyền dữ liệu qua kênh và có lỗi (giả lập)
received_data = punctured_data.copy()
received_data[2][0] = 1 - received_data[2][0]  # Đảo bit hệ thống tại vị trí 2
print("Dữ liệu bị lỗi trong quá trình truyền:", received_data)

# Giải mã Turbo và sửa lỗi
decoded_data = turbo.turbo_decode(received_data, interleaved_data)
print("Dữ liệu giải mã (sau khi sửa lỗi):", decoded_data)

# Kiểm tra dữ liệu gốc và dữ liệu giải mã
if data == decoded_data:
    print("Giải mã và sửa lỗi thành công!")
else:
    print("Giải mã và sửa lỗi thất bại!")


Dữ liệu gốc: [1, 1, 1, 0, 1, 1]
Dữ liệu mã hóa Turbo (1/3): [(1, 1, 1), (1, 1, 0), (1, 0, 1), (0, 1, 1), (1, 0, 0), (1, 1, 0)]
Dữ liệu sau khi đục lỗ: [[1, 1], [1, 0], [1, 1], [0, 1], [1, 0], [1, 0]]
Dữ liệu bị lỗi trong quá trình truyền: [[1, 1], [1, 0], [0, 1], [0, 1], [1, 0], [1, 0]]
Dữ liệu giải mã (sau khi sửa lỗi): [1, 1, 0, 0, 1, 1]
Giải mã và sửa lỗi thất bại!


In [2]:
import numpy as np

# --- Chuyển đổi chéo ---
def interleave(codewords):
    interleaved = []
    for i in range(len(codewords[0])):
        interleaved.append([cw[i] for cw in codewords])
    return np.array(interleaved).flatten()

def deinterleave(codewords, original_length):
    deinterleaved = []
    for i in range(original_length):
        deinterleaved.append(codewords[i::original_length])
    return np.array(deinterleaved).flatten()

# Ví dụ
codewords = np.array([[0, 1, 1, 0, 0, 1, 1 ,1, 0, 1]])

# Chuyển đổi chéo
interleaved_data = interleave(codewords)
print(f"Interleaved: {interleaved_data}")

# Giải mã chuyển đổi chéo
original_length = 1
deinterleaved_data = deinterleave(interleaved_data, original_length)
print(f"Deinterleaved: {deinterleaved_data}")


Interleaved: [0 1 1 0 0 1 1 1 0 1]
Deinterleaved: [0 1 1 0 0 1 1 1 0 1]


In [5]:
data_to_send = [np.random.randint(0, 2, 4) for _ in range(5)]
data_to_send

[array([1, 0, 0, 0]),
 array([0, 1, 0, 0]),
 array([1, 0, 0, 1]),
 array([0, 1, 0, 0]),
 array([1, 1, 0, 0])]

In [ ]:
import time
import threading

# Biến toàn cục để mô phỏng việc gửi và nhận dữ liệu qua mạng
network_buffer = None
ack_received = threading.Event()

def sender(data_packets):
    global network_buffer
    for sequence_number, packet in enumerate(data_packets):
        while True:
            print(f"Sender: Gửi gói tin {sequence_number}")
            network_buffer = (sequence_number, packet)  # Gửi gói tin
            ack_received.clear()  # Reset event cho ACK
            ack_received.wait(timeout=2)  # Đợi ACK với timeout
            
            if ack_received.is_set():
                print(f"Sender: Nhận được ACK cho gói tin {sequence_number}")
                break
            else:
                print(f"Sender: Không nhận được ACK, gửi lại gói tin {sequence_number}")
    print("Sender: Hoàn thành việc gửi tất cả gói tin.")

def receiver():
    global network_buffer
    while True:
        if network_buffer:
            sequence_number, packet = network_buffer
            print(f"Receiver: Nhận được gói tin {sequence_number}")
            time.sleep(1)  # Mô phỏng việc xử lý gói tin
            print(f"Receiver: Gửi ACK cho gói tin {sequence_number}")
            ack_received.set()  # Gửi ACK
            network_buffer = None  # Xóa buffer

# Tạo và khởi chạy các luồng
data_to_send = ['packet1', 'packet2', 'packet3']
receiver_thread = threading.Thread(target=receiver)
sender_thread = threading.Thread(target=sender, args=(data_to_send,))

receiver_thread.start()
sender_thread.start()

sender_thread.join()
receiver_thread.join()

Sender: Gửi gói tin 0
Receiver: Nhận được gói tin 0
Receiver: Gửi ACK cho gói tin 0
Sender: Nhận được ACK cho gói tin 0
Sender: Gửi gói tin 1
Receiver: Nhận được gói tin 1
Receiver: Gửi ACK cho gói tin 1
Sender: Nhận được ACK cho gói tin 1
Sender: Gửi gói tin 2
Receiver: Nhận được gói tin 2
Receiver: Gửi ACK cho gói tin 2
Sender: Nhận được ACK cho gói tin 2
Sender: Hoàn thành việc gửi tất cả gói tin.


In [2]:
import numpy as np
encoded_data = np.array([0, 1, 1, 0, 0, 1, 1, 1, 0, 0], dtype=np.uint8)
byte_data = encoded_data.tobytes()
print(byte_data)  # Output: b'\x03\x18'


b'\x00\x01\x01\x00\x00\x01\x01\x01\x00\x00'
